In [25]:
import pandas as pd

In [26]:
train = pd.read_csv("_train.csv").sample(frac=1)
test = pd.read_csv("_test.csv")

In [27]:
print(train.head())

                                                msno  is_churn  city    bd  \
450244  v2ieMwTcDkFZJsDruMyXmJihejQHkPwNRVySiFf0zzM=         0  13.0   0.0   
533919  1Pnx+jAMJX3NxgaRPaLqI2ZRjXNmF/8ZMAmuGzinKLI=         0  13.0  22.0   
941056  CwRILDASvX9fCvRx7ADuEyndM7UypP//I0HO/DNqMn8=         0   5.0  32.0   
329694  dbdTZv0gV2Fsgel1r6qAqdxrpmk72lvZFYaIYllG+so=         0  15.0  29.0   
10354   t7uJNOsEgOjn+t4HgQvt6TzQ1Rfq8JDSI21BjPNN4J0=         1  13.0  18.0   

        gender  registered_via  registration_init_time  expiration_date  \
450244  -999.0             3.0              20131024.0       20171001.0   
533919     2.0             9.0              20130327.0       20180205.0   
941056     2.0             4.0              20160425.0       20171006.0   
329694  -999.0             3.0              20130515.0       20170912.0   
10354      1.0             9.0              20150509.0       20170716.0   

        trans_count_x  trans_count_y  payment_method_id  payment_plan_days  \
45

In [28]:
from catboost import CatBoostClassifier

In [29]:
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score

# Load data set and target values
data, target = train.drop(['is_churn', 'msno'], axis=1), train[["is_churn"]]

In [32]:

def catboostcv(d, l2, rate, i):
    return cross_val_score(CatBoostClassifier(
        depth=int(d), 
        l2_leaf_reg=int(l2), 
        learning_rate=rate,
        iterations=int(i),
        loss_function='Logloss',
        eval_metric='Logloss',
        random_seed=1337
    ), data, target, "neg_log_loss", cv=5).mean()

catboostBO = BayesianOptimization(catboostcv, 
                             {'d': (1, 15), 
                              'l2': (0, 3), 
                              'rate': (0.01, 0.1), 
                              'i': (10, 1000)})
catboostBO.explore({
    'd': [3, 10], 
    'l2': [2, 3], 
    'rate': [0.03, 0.06], 
    'i': [100, 800]})

catboostBO.maximize()

print('Final Results')
print('Catboost: %f' % catboostBO.res['max']['max_val'])

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |         d |         i |        l2 |      rate | 


/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


    1 | 01m02s |   -0.11987 |    3.0000 |  100.0000 |    2.0000 |    0.0300 | 


KeyboardInterrupt: 

In [33]:
model = CatBoostClassifier(
        depth=3, 
        l2_leaf_reg=2, 
        learning_rate=0.03,
        iterations=100,
        loss_function='Logloss',
        eval_metric='Logloss',
        random_seed=1337)

In [34]:
model.fit(
        train.drop(['is_churn', 'msno'], axis=1), train[["is_churn"]],
        verbose = True)

Borders for float features generated
0:	learn 0.6400443805	total: 358ms	remaining: 35.5s
1:	learn 0.5895722058	total: 681ms	remaining: 33.4s
2:	learn 0.5464364203	total: 981ms	remaining: 31.7s
3:	learn 0.5135372389	total: 1.29s	remaining: 31s
4:	learn 0.4765809328	total: 1.6s	remaining: 30.4s
5:	learn 0.4488618271	total: 1.92s	remaining: 30s
6:	learn 0.4241047027	total: 2.23s	remaining: 29.6s
7:	learn 0.3982478463	total: 2.52s	remaining: 29s
8:	learn 0.3733652588	total: 2.81s	remaining: 28.4s
9:	learn 0.3525119091	total: 3.11s	remaining: 28s
10:	learn 0.3336478538	total: 3.4s	remaining: 27.6s
11:	learn 0.317881064	total: 3.72s	remaining: 27.3s
12:	learn 0.3035184432	total: 4.02s	remaining: 26.9s
13:	learn 0.2904304099	total: 4.34s	remaining: 26.7s
14:	learn 0.2784159503	total: 4.65s	remaining: 26.4s
15:	learn 0.267524256	total: 4.95s	remaining: 26s
16:	learn 0.2580970994	total: 5.24s	remaining: 25.6s
17:	learn 0.2461166607	total: 5.53s	remaining: 25.2s
18:	learn 0.2386810143	total: 5.8

In [35]:
# print(test.head())
result = model.predict(test.drop(["msno", "is_churn"], axis = 1))

In [36]:
test[["is_churn"]] = result

In [37]:
(test[["msno", "is_churn"]]).to_csv("submission.csv", index = False)